### Exercício 3

Você deverá selecionar um banco de dados científico aberto entre as seguintes opções para realizar uma análise visual aprofundada, utilizando Python:

- PANGAEA (dados ambientais e climáticos)
- NASA Earth Data (observações de satélite)
- ENSEMBL / 1000 Genomes (dados genômicos)
- Global Biodiversity Information Facility (GBIF)
- UN Data (indicadores globais socioeconômicos)
- Figshare ou Zenodo (dados multidisciplinares de pesquisas científicas)

🎯 Tarefas:
1. Escolha um tema exclusivo dentro do banco de dados escolhido, que ainda não tenha sido selecionado por outro colega (registre seu tema antes de iniciar).
2. Baixe e faça a limpeza dos dados, garantindo organização para análise.
3. Utilize Python para produzir ao menos cinco visualizações diferentes, incluindo obrigatoriamente:
    - Gráfico de linha ou série temporal (quando aplicável)
    - Gráfico de dispersão ou correlação
    - Gráfico de distribuição (histograma, boxplot ou violin plot)
    - Visualização geoespacial (mapa estático ou interativo) — caso os dados tenham componente geográfico
    - Visualização avançada ou interativa à sua escolha (ex: gráfico de rede, heatmap, gráfico radial, dashboard simples)

4. Escreva um relatório científico (mínimo 500 palavras) contendo:
    - Contextualização do tema e relevância científica
    - Principais insights obtidos nas visualizações
    - Discussão crítica das limitações dos dados e da análise
    - Referências das bases de dados utilizadas


🛑 Regras Importantes:

- Cada aluno deve escolher um tema e banco de dados exclusivos.
- O código Python deve ser organizado, comentado e permitir reprodução da análise.
- As visualizações devem ser claras, esteticamente cuidadas e com legendas/títulos informativos.
- O relatório deve demonstrar domínio conceitual e capacidade de comunicação científica.

### TEMA: Salinidade Oceânica Global

In [ ]:
1. Download and clean the data from the World Ocean Atlas 2023 for the ocean salinity. The .csv file is found at `https://www.ncei.noaa.gov/data/oceans/woa/WOA23/DATA/salinity/csv/decav/5.00/woa23_s_decav_5.00_csv.tar.gz` (as far as I know);
2. Use Python to produce at least five different visualizations, including:
- Line or time series graph (when applicable)
- Scatter or correlation graph
- Distribution graph (histogram, boxplot or violin plot)
- Geospatial visualization (static or interactive map) — if the data has a geographic component
- Advanced or interactive visualization of your choice (e.g.: network graph, heatmap, radial graph, simple dashboard)

3. Write a scientific report (minimum 500 words) containing:
- Contextualization of the topic and scientific relevance
- Main insights obtained from the visualizations
- Critical discussion of the limitations of the data and analysis
- References of the databases used